In [1]:
import pandas as pd
import sqlite3

In [2]:
def load_data(filename="mxm_dataset_train copy.txt"):
    df = pd.read_csv(filename, names=['Lyrics'],sep="\n",index_col=None)
    df['ID'] = df['Lyrics'].apply(lambda x: x.split(",")[0])
    df['MXM'] = df['Lyrics'].apply(lambda x: x.split(",")[1])
    df['Lyrics'] = df['Lyrics'].apply(lambda x: x.split(",")[2:])
    df = df[['ID',"MXM","Lyrics"]]
    
    return df

In [3]:
def getWords(filename="Words.txt"):
    d = {}
    f = open(filename)
    for line in f:
        words = line.split(",")
        for i, word in enumerate(words):
            d[i+1] = word
    return d

In [4]:
train = load_data("/content/drive/MyDrive/Music Recommendation/mxm_dataset_train copy.txt")

In [5]:
words = getWords("/content/drive/MyDrive/Music Recommendation/Words.txt")

In [6]:
def getLyrics(words, embeddings, with_count = False):
    sentence = ""
    for emb in embeddings:
        ids = emb.split(":")
        #print(ids)
        #ids = map(int, ids)
        sentence += words[int(ids[0])] + " "
    return sentence
        

In [7]:
train['Sentence'] = train['Lyrics'].apply(lambda x: getLyrics(words, x))

In [9]:
sentence_list = train['Sentence'].values.tolist() 


In [29]:
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.2 MB/s 
     |████████████████████████████████| 1.3 MB 32.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=8323e1ec4b4bb1d39a7ab79e8df695c138ec2c2372bebe0a7c1e3bcce664b5cd
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [41]:
sentence_embeddings = model.encode(sentence_list[:1])
sentence_embeddings

array([[-1.73030376e-01,  6.05959237e-01,  3.28566492e-01,
         5.69573522e-01,  5.55066049e-01, -4.18390930e-01,
        -3.02619487e-01,  7.72664696e-02,  6.90237164e-01,
        -4.05103832e-01, -1.31111294e-01,  1.10774803e+00,
         3.09345901e-01,  2.43669465e-01,  3.31137657e-01,
         8.94251943e-01, -5.30155063e-01, -6.90041184e-02,
         5.18728733e-01, -5.04419029e-01, -4.97019529e-01,
        -1.51450709e-01,  8.57545715e-03,  5.76746464e-01,
         1.38209891e+00,  7.69639850e-01,  4.78905112e-01,
         2.46822685e-02, -8.07275593e-01,  8.65446448e-01,
         6.25050962e-02,  4.68196809e-01, -3.53738397e-01,
        -1.33936137e-01, -5.13407230e-01,  5.60669124e-01,
        -3.60197186e-01, -2.68362254e-01,  2.37626627e-01,
         4.96273816e-01,  2.40691274e-01, -5.08985147e-02,
         6.09996974e-01,  3.03627811e-02, -9.73035634e-01,
         6.99883252e-02,  1.80295438e-01,  6.18495822e-01,
         3.04521203e-01, -3.02433640e-01,  7.50923336e-0

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
cosine_similarity(
    [sentence_embeddings[0]],
    sentence_embeddings[1:]
)

array([[0.59449977, 0.624264  , 0.57367456, 0.622557  , 0.67812955,
        0.52390873, 0.736699  , 0.66215503, 0.67828846, 0.5696923 ,
        0.6730179 , 0.68565464, 0.6606122 , 0.5702472 , 0.7261603 ,
        0.7034421 , 0.5745092 , 0.72129   , 0.63918364, 0.76486504,
        0.6359271 , 0.7292168 , 0.64341307, 0.73327374, 0.6007491 ,
        0.67163086, 0.6612719 , 0.63356   , 0.6488082 ]], dtype=float32)

In [42]:
train['Encodings'] = train['Sentence'].apply(lambda x: model.encode(x))

In [46]:
train['Encodings'].to_pickle("/content/drive/MyDrive/Music Recommendation/Encodings.pickle")

In [1]:
import pandas as pd
emb = pd.read_pickle("/content/drive/MyDrive/Music Recommendation/Encodings.pickle")

In [48]:
train.head()

,ID,MXM,Lyrics,Sentence,Encodings
0,TRAAAAV128F421A322,4623710,"[1:6, 2:4, 3:2, 4:2, 5:5, 6:3, 7:1, 8:1, 11:1,...",i the you to and a me it my is of your that ar...,"[-0.17303038, 0.60595924, 0.3285665, 0.5695735..."
1,TRAAABD128F429CF47,6477168,"[1:10, 3:17, 4:8, 5:2, 6:2, 7:1, 8:3, 9:2, 10:...",i you to and a me it not in my is your that do...,"[0.4436474, 0.41428164, 1.8322037, 0.09133685,..."
2,TRAAAED128E0783FAB,2516445,"[1:28, 2:15, 3:2, 4:12, 5:22, 6:2, 7:2, 8:4, 9...",i the you to and a me it not in my is of that ...,"[-0.045117036, 0.14868477, 1.5242851, 0.115475..."
3,TRAAAEF128F4273421,3759847,"[1:5, 2:4, 3:3, 4:2, 5:1, 6:11, 9:4, 12:9, 13:...",i the you to and a not is of that do are for k...,"[-0.57483697, 0.6301075, 0.8557763, 0.10567400..."
4,TRAAAEW128F42930C0,3783760,"[1:4, 4:5, 5:7, 6:2, 7:4, 9:1, 10:1, 11:9, 15:...",i to and a me not in my that on am all with li...,"[0.34970948, 1.2279826, 0.17302203, 0.34280318..."


In [3]:
emb

0         [-0.17303038, 0.60595924, 0.3285665, 0.5695735...
1         [0.4436474, 0.41428164, 1.8322037, 0.09133685,...
2         [-0.045117036, 0.14868477, 1.5242851, 0.115475...
3         [-0.57483697, 0.6301075, 0.8557763, 0.10567400...
4         [0.34970948, 1.2279826, 0.17302203, 0.34280318...
                                ...                        
210514    [-0.28232738, 0.062059842, 1.3693401, 0.419777...
210515    [-0.15344067, 0.76832426, -0.13653128, 0.25248...
210516    [-0.018554613, 0.58642435, 1.627772, 0.2877465...
210517    [-0.04823925, 0.8533982, 0.98554724, 0.31326, ...
210518    [-0.055622995, 0.86753625, 0.81155956, 0.41680...
Name: Encodings, Length: 210519, dtype: object